In [50]:
%pprint

Pretty printing has been turned OFF


In [51]:
from src.core.SDR import *

Create an SDR with a range of 32

In [52]:
sdr = SDR(32)
print("Binary Array: ",sdr.toBinaryArray())
print("Index Array: ",sdr.toIndexArray())
print("Density: " ,sdr.density())
print("Sparsity: " ,sdr.sparsity())

Binary Array:  [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0]
Index Array:  [3, 4, 9, 16, 18, 24, 27, 28]
Density:  0.25
Sparsity:  0.75


The array above is all the indices of the on bits in the SDR

The sparsity of the SDR above is very low. SDRs work better when they have a greater range and sparsity.

In [53]:
sdr.range = 1024
sdr.random(16) # create new active bits, randomly
print(sdr.toIndexArray())
print("Density: " ,sdr.density())
print("Sparsity: " ,sdr.sparsity())

[60, 83, 130, 228, 307, 362, 423, 493, 555, 612, 682, 720, 809, 857, 928, 999]
Density:  0.015625
Sparsity:  0.984375


### Operations

**Subsample** - Evenly remove active bits from the SDR

In [54]:
print(sdr.subsample(8)) # subsample down to a size of 8 bits

[60, 130, 307, 423, 555, 682, 809, 928]


**Union** - Combine multiple SDRs

In [55]:
sdr.union([SDR().indices,SDR().indices])

[60, 83, 130, 228, 307, 362, 423, 493, 555, 612, 682, 720, 809, 857, 928, 999, 21, 286, 639, 824, 1120, 1404, 1550, 1992, 15, 307, 643, 793, 1093, 1460, 1677, 1907]

**Overlap** - Matches the indices (on bits) that exist in both SDRs

In [56]:
sdr.indices = [63, 190, 318, 429, 554, 647, 773, 924] # reassignment, for ease of demonstration
print(sdr.overlap([63, 113, 318, 417, 554, 647, 784]))

[63, 318, 554, 647]


**Subtract** - Removes the matching indices between the two SDRs (the opposite of overlap)

In [57]:
print(sdr.subtract([63, 113, 318, 417, 554, 647, 784]))

[190, 429, 773, 924]


**Difference** - Combines the SDRs and removes the overlap

In [58]:
print(sdr.difference([63, 113, 318, 417, 554, 647, 784]))

[190, 429, 773, 924, 113, 417, 784]


## SDRMap

In [59]:
from src.core.SDRMap import *
map = SDRMap()

An SDRMap is a key-value mapping data structure. It maps SDRs using their indices.

In [60]:
map.set([190, 429, 773, 924, 113, 417, 784, 941],[63, 107, 318, 487, 554, 614, 758, 860])
print(map.get([190, 429, 773, 924, 113, 417, 784, 941]))

[63, 107, 318, 487, 554, 614, 758, 860]


The indices from the key SDR are used to store & retreive the value SDR. Unlike a traditional key-value data structures, the key can be noisy and/or incomplete.

In [61]:
print(map.get([190, 773, 113, 417, 941])) # missing 3 bits
print(map.get([190, 110, 773, 924, 100, 417, 784, 900])) # 3 incorrect bits
print(map.get([429, 924, 784, 941, 417, 113, 773, 190])) # bits in difference order
print(map.get([190, 429, 773, 847, 924, 528, 1197, 113, 417, 784, 941, 1200])) # 4 extra bits

[63, 107, 318, 487, 554, 614, 758, 860]
[63, 107, 318, 487, 554, 614, 758, 860]
[63, 107, 318, 487, 554, 614, 758, 860]
[63, 107, 318, 487, 554, 614, 758, 860]


All the keys above retrieved the value successfully. However, the key invariance has a threshold. If the key does not meet that threshold then the value cannot be retreived.

In [62]:
print(map.get([429, 924, 417, 784])) # 4 missing bits
print(map.get([29, 429, 41, 114, 580, 417, 784, 941])) # 4 incorrect bits

[]
[]


The threshold for the above map is 0.5, the size of the key is 8, so therefore the number of correct key indices has to be above 4 (8 * 0.5 = 4). The threshold can be changed.

In [63]:
map.threshold = 0.4 # correct key indices must be above 3.2
print(map.get([429, 924, 417, 784]))
print(map.get([429, 924, 417]))
map.threshold = 0.1 # above 0.8
print(map.get([429]))
print(map.get([]))
map.threshold = 0.7 # above 5.6
print(map.get([190, 429, 773, 924, 113, 417]))
print(map.get([190, 429, 924, 113, 417]))

[63, 107, 318, 487, 554, 614, 758, 860]
[]
[63, 107, 318, 487, 554, 614, 758, 860]
[]
[63, 107, 318, 487, 554, 614, 758, 860]
[]


The usefulness of this property is that a subsampled SDR key can be used to retreive the value.

In [64]:
subsample = SDR.Subsample([190, 429, 773, 924, 113, 417, 784, 941],5) # subsample down to a size of 5
print(subsample)
map.threshold = 0.5
print(map.get(subsample))

[113, 190, 417, 429, 773]
[63, 107, 318, 487, 554, 614, 758, 860]


The same is true if the key is a part of a union.

In [65]:
union = SDR.Union([[190, 429, 773, 924, 113, 417, 784, 941],SDR().indices])
print(union)
print(map.get(union))

[190, 429, 773, 924, 113, 417, 784, 941, 235, 505, 618, 922, 1224, 1452, 1781, 1912]
[63, 107, 318, 487, 554, 614, 758, 860]


Or subsampled and within a union.

In [66]:
union = SDR.Union([SDR.Subsample([190, 429, 773, 924, 113, 417, 784, 941],5),SDR().indices])
print(union)
print(map.get(union))

[113, 190, 417, 429, 773, 88, 488, 756, 842, 1224, 1424, 1638, 1863]
[63, 107, 318, 487, 554, 614, 758, 860]


SDRMap can have more than just a one-to-one key-value mapping.

In [67]:
map = SDRMap()
print('many-to-one relationship')
map.set([168, 477, 608, 803, 1047, 1400, 1543, 1997],[63, 107, 318, 487, 554, 614, 758, 860])
map.set([208, 383, 662, 870, 1038, 1321, 1646, 1834],[63, 107, 318, 487, 554, 614, 758, 860])
print(map.get([168, 477, 608, 803, 1047, 1400, 1543, 1997]))
print(map.get([208, 383, 662, 870, 1038, 1321, 1646, 1834]))
print('one-to-many relationship')
map = SDRMap()
map.set([146, 327, 762, 821, 1268, 1353, 1790, 1832],[86, 422, 529, 1021, 1197, 1321, 1541, 1818])
map.set([146, 327, 762, 821, 1268, 1353, 1790, 1832],[32, 395, 599, 937, 1046, 1497, 1540, 1850])
print(map.get([146, 327, 762, 821, 1268, 1353, 1790, 1832])) # union of mapped values
print('many-to-many relationship')
map = SDRMap()
map.set([50, 427, 546, 843, 1260, 1436, 1654, 1979],[233, 385, 711, 899, 1239, 1381, 1622, 1870])
map.set([50, 427, 546, 843, 1260, 1436, 1654, 1979],[1, 445, 591, 806, 1215, 1522, 1647, 1955])
map.set([235, 478, 705, 817, 1169, 1441, 1702, 1851],[233, 385, 711, 899, 1239, 1381, 1622, 1870])
map.set([235, 478, 705, 817, 1169, 1441, 1702, 1851],[1, 445, 591, 806, 1215, 1522, 1647, 1955])
print(map.get([50, 427, 546, 843, 1260, 1436, 1654, 1979]))
print(map.get([235, 478, 705, 817, 1169, 1441, 1702, 1851]))

many-to-one relationship
[63, 107, 318, 487, 554, 614, 758, 860]
[63, 107, 318, 487, 554, 614, 758, 860]
one-to-many relationship
[86, 422, 529, 1021, 1197, 1321, 1541, 1818, 32, 395, 599, 937, 1046, 1497, 1540, 1850]
many-to-many relationship
[233, 385, 711, 899, 1239, 1381, 1622, 1870, 1, 445, 591, 806, 1215, 1522, 1647, 1955]
[233, 385, 711, 899, 1239, 1381, 1622, 1870, 1, 445, 591, 806, 1215, 1522, 1647, 1955]
